<a href="https://colab.research.google.com/github/Romanchenko-RS/ML/blob/main/%D0%9A%D0%BE%D0%BB%D0%BB%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D0%A0%D0%BE%D0%BC%D0%B0%D0%BD%D1%87%D0%B5%D0%BD%D0%BA%D0%BE_%D0%A0_%D0%A1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Импорт необходмых библиотек**

In [ ]:
import pandas as pd
from surprise.model_selection import train_test_split
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.accuracy import rmse
from surprise.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'surprise'

**Получим данные**

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2024-06-25 06:08:32--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 142.251.31.101, 142.251.31.138, 142.251.31.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.31.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-06-25 06:08:32--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.127.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.01s   

2024-06-25 06:08:34 (60.2 MB/s)

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


**Установим библиотку SURPRISE**

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357252 sha256=bbefb700b36236a7e08d18b65b5175d25385f5bbe06b025a5f0afcbc61a8bf3d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


**Формируем датасет**

In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
# data_1 = Dataset.load_from_df(movies_with_ratings, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

**Применим алгоритм SVD**

In [ ]:
algo = SVD()

In [ ]:
# Обучение алгоритма
algo.fit(trainset)

In [ ]:
# Предсказания на тестовом наборе
predictions = algo.test(testset)

In [ ]:
# Вычисление RMSE
accuracy = rmse(predictions)

RMSE: 0.8753


In [ ]:
# Создание словаря для записи результатов
results = pd.DataFrame(columns=['Метод', 'RMSE'])

In [ ]:
results.loc[len(results)] = ['Базовая модель', accuracy]

In [ ]:
results

,Метод,RMSE
0,Базовая модель,0.875283


**Настроим кросс-валидацию**

In [ ]:
# Настройка кросс-валидации
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8641  0.8755  0.8777  0.8761  0.8801  0.8747  0.0055  
Fit time          1.99    1.87    2.67    1.75    1.80    2.02    0.34    
Test time         0.13    0.26    0.21    0.30    0.12    0.21    0.07    


{'test_rmse': array([0.8641257 , 0.87546314, 0.87770295, 0.87606723, 0.88014523]),
 'fit_time': (1.9867355823516846,
  1.8746073246002197,
  2.669801712036133,
  1.7543892860412598,
  1.8037872314453125),
 'test_time': (0.13186097145080566,
  0.26151180267333984,
  0.2145676612854004,
  0.3011343479156494,
  0.12497854232788086)}

Как видим, базовые настройки гиперпараметров дают среднне значение метрики равное 0.8747, что является выше требуемого. Произведём поиск наилучших гиперпараметров. Для этого используем метод GridSearchCV

In [ ]:
# Задаём настройки для GridSearch
param_grid = {
    'n_epochs': [5, 10, 20],  # Количество эпох
    'lr_all': [0.002, 0.005, 0.01],  # Скорость обучения
    'reg_all': [0.02, 0.1, 0.4]  # Коэффициент регуляризации
}

In [ ]:
# Инициализация GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)

In [ ]:
# Начало обучения
gs.fit(data)

In [ ]:
# Результаты - лучшие параметры и результаты
print("Лучшие параметры: ", gs.best_params['rmse'])
print("Лучший RMSE: ", gs.best_score['rmse'])

# Использование лучшей модели
best_model = gs.best_estimator['rmse']

Лучшие параметры:  {'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.1}
Лучший RMSE:  0.8623965740363273


In [ ]:
results.loc[len(results)] = ['Лучшая модель по анализу сетки', gs.best_score['rmse']]

**Вывод**

In [ ]:
results

,Метод,RMSE
0,Базовая модель,0.875283
1,Лучшая модель по анализу сетки,0.862397


Благодаря кросс-валидации и поиску по сетке удаётся получить модель с лучшим качеством, чем при использовании базовых настроек.